# Collision Avoidance - Live Demo

In this notebook we'll use the model we trained to detect whether the robot is ``free`` or ``blocked`` to enable a collision avoidance behavior on the robot.  


#### [realImg ~> unet ~> change color (back, obs, path) ~> qnet ~> robot control]

## Load the trained model

We'll assumed that you've already downloaded the ``best_model.pth`` to your workstation as instructed in the training notebook.  Now, you should upload this model into this notebook's
directory by using the Jupyter Lab upload tool.  Once that's finished there should be a file named ``best_model.pth`` in this notebook's directory.  

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [1]:
# import torch
# import torchvision

# model = torchvision.models.alexnet(pretrained=False)
# model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

Next, load the trained weights from the ``best_model.pth`` file that you uploaded

In [13]:
# model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [14]:
# device = torch.device('cuda')
# model = model.to(device)

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [ ]:
import os
import argparse
import sys
import time
import signal
import cv2
import numpy as np

import torch.nn.functional as F
import torchvision.transforms.functional as TF
import torchvision
import torch
from torchvision import transforms
from PIL import Image

from unet import UNet
from jetbotSim import Robot, Env
from rl import dqn, models


In [ ]:
def preprocess(state, img_size, th=0.4):
    # x = state
    # state = cv2.cvtColor(state, cv2.COLOR_BGR2RGB)
    state = np.array(Image.fromarray(state).resize(img_size, Image.Resampling.BILINEAR))
    state = state.astype(float) / 255.
    state = state.transpose(2,0,1)
    return state

def step(action):
    global robot
    if action == 0:
        robot.set_motors(0.2, 0.2)
    elif action == 1:
        robot.set_motors(0.1, 0.)
    elif action == 2:
        robot.set_motors(0., 0.1)

In [ ]:
def robot_control(mask):
    # img = mask
    # flippedImg = cv2.flip(img, 0)

    return mask==1, mask==2

def predict_img(full_img, out_threshold=0.5):
    global transform, device, net
    r, g, b = full_img.split()
    img = Image.merge("RGB", (b, g, r))
    img = transform(img)
    img = img.unsqueeze(0).to(device=device)
    with torch.no_grad():
        output = net(img).cpu()
        output = F.interpolate(output, (full_img.size[1], full_img.size[0]), mode='bilinear')
        if net.n_classes > 1:
            mask = output.argmax(dim=1)
        else:
            mask = torch.sigmoid(output) > out_threshold

    return mask[0].long().squeeze().numpy()

def process_unet_img(image_bgr):
    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    # Create a PIL Image object from the RGB image
    ori_img = Image.fromarray(image_rgb)

    mask = predict_img(ori_img)
    classes = mask.max() + 1

    # Convert the PIL Image object to a NumPy array
    # if classes >1:
    obs_mask = np.array(mask==2)
    back_mask = np.array(mask==0)
    path_mask = np.array(mask==1)

    # Create a blank color image
    colored_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)

    # Set the colors for each mask
    colored_mask[back_mask] = [185, 205, 235]  # skyblue
    colored_mask[path_mask] = [26, 28, 31]     # black
    colored_mask[obs_mask] = [170, 40, 40]     # red

    return colored_mask

In [15]:
# import cv2
# import numpy as np

# mean = 255.0 * np.array([0.485, 0.456, 0.406])
# stdev = 255.0 * np.array([0.229, 0.224, 0.225])

# normalize = torchvision.transforms.Normalize(mean, stdev)

# def preprocess(camera_value):
#     global device, normalize
#     x = camera_value
#     x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
#     x = x.transpose((2, 0, 1))
#     x = torch.from_numpy(x).float()
#     x = normalize(x)
#     x = x.to(device)
#     x = x[None, ...]
#     return x

In [ ]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])
normalize = torchvision.transforms.Normalize(mean, stdev)

qnet_model_dir = "./rl_checkpoint/"
qnet_model_name = "qnet_480000.pt"
model_path = "./unet/checkpoints/checkpoint_epoch4.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

unet_model_path = "./unet/checkpoints/checkpoint_epoch5.pth"

frames = -1
total_reward = 0
stack_frames = 3
img_size = (128, 128)

recent_distances = []  # List to store recent distance measurements
MAX_HISTORY = 4

net = None

In [ ]:
def initialize_all(model_dir, model_name, unet_model_path):
  global total_reward, stack_frames, img_size, device
  print(f"Device: {device}")

  # Load UNET
  net = UNet(n_channels=3, n_classes=3, bilinear=False)
  net.to(device=device)
  state_dict = torch.load(model_path, map_location=device)
  mask_values = state_dict.pop('mask_values', [0, 1, 2])
  net.load_state_dict(state_dict)
  net.eval()

  # Load QNET
  agent = dqn.DeepQNetwork(
      n_actions = 3,
      input_shape = [(stack_frames)*3, *img_size],
      qnet = models.QNet,
      device = device,
      learning_rate = 2e-4, 
      reward_decay = 0.98,
      replace_target_iter = 1000, 
      memory_size = 10000,
      batch_size = 32,
  )
  agent.save_load_model("load", path=model_dir, fname=model_name)
  
  return net, agent


In [ ]:


net, agent = initialize_all(qnet_model_dir, qnet_model_name, unet_model_path)

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

In [18]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot, frames, stack_frames, img_size
    global state, state_dict
    # x = change['new'] 
    # x = preprocess(x)
    # y = model(x)
    
    # # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    # y = F.softmax(y, dim=1)
    
    # prob_blocked = float(y.flatten()[0])
    
    # blocked_slider.value = prob_blocked
    
    # if prob_blocked < 0.5:
    #     robot.forward(0.4)
    # else:
    #     robot.left(0.4)

    frames += 1

    state_next = change['new']
    state_next = process_unet_img(state_next)

    if frames != 0:

        # Select action.
        action = agent.choose_action(state_dict, 0.2)

        # Get next stacked state.
        step(action)
        state_next = preprocess(state_next, img_size=img_size)
        state_next = np.concatenate([state_next, state[3:]], 0)
        state_next_dict = {"obs":state_next}
        # total_reward += reward
        state = state_next.copy()
        state_dict = state_next_dict

    else:
        frames = 0
        # total_reward = 0
        state = preprocess(state_next, img_size=img_size)
        state = state.repeat(stack_frames, axis=0)
        state_dict = {"obs":state}
        # robot.reset()
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

We'll also create our robot instance which we'll need to drive the motors.

In [17]:
from jetbot import Robot

robot = Robot()

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing. 

We accomplish that with the ``observe`` function.

> WARNING: This code will move the robot!! Please make sure your robot has clearance.  The collision avoidance should work, but the neural
> network is only as good as the data it's trained on!

In [21]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame.  Perhaps start by placing your robot on the ground and seeing what it does when it reaches an obstacle.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [22]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

Perhaps you want the robot to run without streaming video to the browser.  You can unlink the camera as below.

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

To continue streaming call the following.

In [ ]:
camera_link.link()  # stream to browser (wont run camera)

### Conclusion

That's it for this live demo!  Hopefully you had some fun and your robot avoided collisions intelligently! 

If your robot wasn't avoiding collisions very well, try to spot where it fails.  The beauty is that we can collect more data for these failure scenarios
and the robot should get even better :)